In [1]:
import pandas as pd
import psycopg2
import sqlite3

In [60]:
sl_conn = sqlite3.connect('titanic.sqlite3')
sl_curs = sl_conn.cursor()

In [111]:
dbname = 'jzycbpdj'
user = 'jzycbpdj'
password = '1mJjvh-9Gz5Y5PJpjXfkEaehYguEKWwz'  # Don't commit this!
host = 'raja.db.elephantsql.com'
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)
pg_curs = pg_conn.cursor()

In [112]:
# we can query tables if we want to check
# this is a clever optional thing, showing postgresql internals
show_tables = '''
SELECT * FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
'''
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'jzycbpdj', None, True, False, False, False),
 ('public',
  'charactercreator_character',
  'jzycbpdj',
  None,
  True,
  False,
  False,
  False),
 ('public', 'titanic_table', 'jzycbpdj', None, True, False, False, False)]

In [107]:
pg_curs.execute('SELECT * FROM titanic_table;')
pg_curs.fetchone()

(1, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [70]:
# Get column names
pg_curs.execute('SELECT * FROM titanic_table')
next(zip(*pg_curs.description))

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


### How many passengers survived, and how many died?

- 342 survived
- 545 died

In [45]:
query = '''
SELECT survived, COUNT(*) AS num
FROM titanic_table
GROUP BY survived;
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(0, 545), (1, 342)]

### How many passengers were in each class?

- 1st class - 216 passengers
- 2nd class - 184 passengers
- 3rd class - 487 passengers

In [46]:
query = '''
SELECT pclass, COUNT(*) AS num
FROM titanic_table
GROUP BY pclass;
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 216), (3, 487), (2, 184)]

### How many passengers survived/died within each class?

- 1st class - 136 survived, 80 died
- 2nd class - 87 survived, 97 died
- 3rd class - 119 survived, 368 died

In [47]:
query = '''
SELECT pclass, COUNT(*) AS num
FROM titanic_table
WHERE survived = 1
GROUP BY pclass;
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 136), (3, 119), (2, 87)]

In [48]:
query = '''
SELECT pclass, COUNT(*) AS num
FROM titanic_table
WHERE survived = 0
GROUP BY pclass;
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 80), (3, 368), (2, 97)]

### What was the average age of survivors vs nonsurvivors?

- Survivors - 28.4 avg age
- Nonsurvivors - 30.14 avg age

In [64]:
query = '''
SELECT survived, avg(age)
FROM titanic_table
GROUP BY survived;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(0, 30.13853211009174), (1, 28.408391812865496)]

### What was the average age of each passenger class?

- Class 1 - 38.79 avg age
- Class 2 - 29.87 avg age
- Class 3 - 25.19 avg age

In [66]:
query = '''
SELECT pclass, avg(age)
FROM titanic_table
GROUP BY pclass;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(1, 38.78898148148148), (2, 29.868641304347825), (3, 25.188747433264886)]

### What was the average fare by passenger class?

- Class 1 - 84.15 avg fare
- Class 2 - 20.66 avg fare
- Class 3 - 13.71 avg fare

In [67]:
query = '''
SELECT pclass, avg(fare)
FROM titanic_table
GROUP BY pclass;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(1, 84.15468749999992), (2, 20.66218315217391), (3, 13.707707392197129)]

### What was the average fare by survival?

- Survivors - 48.40 avg fare
- Nonsurvivors - 22.21 avg fare

In [68]:
query = '''
SELECT survived, avg(fare)
FROM titanic_table
GROUP BY survived;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(0, 22.208584036697225), (1, 48.39540760233917)]

### How many siblings/spouses aboard on average, by passenger class?

- Class 1 - 0.42 avg siblings/spouses aboard
- Class 2 - 0.40 avg siblings/spouses aboard
- Class 3 - 0.62 avg siblings/spouses aboard

In [77]:
query = '''
SELECT pclass, avg(`Siblings/Spouses Aboard`)
FROM titanic_table
GROUP BY pclass;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(1, 0.4166666666666667), (2, 0.40217391304347827), (3, 0.6201232032854209)]

### How many siblings/spouses aboard on average, by survival?

- Survivors - 0.47 avg siblings/spouses aboard
- Nonsurvivors - 0.56 avg siblings/spouses aboard

In [78]:
query = '''
SELECT survived, avg(`Siblings/Spouses Aboard`)
FROM titanic_table
GROUP BY survived;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(0, 0.5577981651376147), (1, 0.47368421052631576)]

### How many parents/children aboard on average, by passenger class?

- Class 1 - 0.36 avg parents/children aboard
- Class 2 - 0.38 avg parents/children aboard
- Class 3 - 0.40 avg parents/children aboard

In [79]:
query = '''
SELECT pclass, avg(`Parents/Children Aboard`)
FROM titanic_table
GROUP BY pclass;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(1, 0.35648148148148145), (2, 0.3804347826086957), (3, 0.39630390143737165)]

### How many parents/children aboard on average, by survival?

- Survivors - 0.46 avg parents/children aboard
- Nonsurvivors - 0.33 avg parents/children aboard

In [80]:
query = '''
SELECT survived, avg(`Parents/Children Aboard`)
FROM titanic_table
GROUP BY survived;
'''
sl_curs.execute(query)
sl_curs.fetchall()

[(0, 0.3321100917431193), (1, 0.4649122807017544)]

### Do any passengers have the same name?

In [108]:
query = '''
SELECT SUM(total.ct)
FROM (
SELECT table_a.first_name, count(*) as ct
FROM (
SELECT split_part(name, ' ', 2) AS first_name
FROM titanic_table
) AS table_a
GROUP BY table_a.first_name
HAVING COUNT(*) > 1
) AS total
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(Decimal('574'),)]

### Do any passengers have the same first name?

In [113]:
query = '''
SELECT a.first_name, count(*)
FROM (SELECT split_part(name, ' ', 2) AS first_name
FROM titanic_table) as a
GROUP BY a.first_name
HAVING COUNT(*) > 1
'''
pg_curs.execute(query)
pg_curs.fetchall()

[('Edward', 13),
 ('Emil', 3),
 ('Dickinson', 2),
 ('Rene', 3),
 ('Washington', 2),
 ('Susan', 2),
 ('Henry', 15),
 ('Josef', 2),
 ('Maurice', 2),
 ('August', 4),
 ('Johan', 10),
 ('Robert', 6),
 ('Hanna', 3),
 ('Elizabeth', 6),
 ('Bertram', 2),
 ('Victor', 6),
 ('Augusta', 3),
 ('Edvard', 2),
 ('Charles', 16),
 ('Lalio', 2),
 ('Walter', 5),
 ('Lillian', 2),
 ('Edgar', 2),
 ('Sinai', 2),
 ('James', 15),
 ('Daniel', 4),
 ('Benjamin', 4),
 ('Samuel', 9),
 ('Frederic', 2),
 ('Hugh', 2),
 ('Elsie', 2),
 ('Johannes', 2),
 ('Richard', 11),
 ('Andrew', 2),
 ('Leonard', 2),
 ('Thomas', 18),
 ('Hanora', 3),
 ('Nils', 4),
 ('Eugene', 2),
 ('Jacob', 2),
 ('Ernst', 5),
 ('William', 48),
 ('Viktor', 2),
 ('Harold', 2),
 ('David', 4),
 ('Madeleine', 2),
 ('Patrick', 4),
 ('Juha', 3),
 ('Bridget', 2),
 ('Jakob', 3),
 ('Norman', 2),
 ('Anna', 9),
 ('Juho', 2),
 ('Mauritz', 2),
 ('Constance', 2),
 ('Ivan', 4),
 ('Jean', 3),
 ('Timothy', 2),
 ('Reginald', 3),
 ('Frederick', 13),
 ('Ernest', 6),
 ('Anton

In [71]:
# Get column names
sl_curs.execute('SELECT * FROM titanic_table')
next(zip(*sl_curs.description))

('index',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare')

In [101]:
sl_curs.execute('PRAGMA table_info(titanic_table)')
sl_curs.fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'Survived', 'INTEGER', 0, None, 0),
 (2, 'Pclass', 'INTEGER', 0, None, 0),
 (3, 'Name', 'TEXT', 0, None, 0),
 (4, 'Sex', 'TEXT', 0, None, 0),
 (5, 'Age', 'REAL', 0, None, 0),
 (6, 'Siblings/Spouses Aboard', 'INTEGER', 0, None, 0),
 (7, 'Parents/Children Aboard', 'INTEGER', 0, None, 0),
 (8, 'Fare', 'REAL', 0, None, 0)]